# Class level relationships

## Generalization

Here, we are going to parse a whole project instead of a single python file.

In [ ]:
# Clone a target project. Today, we try `httpie/cli`, a commandline HTTP client.
!git clone git@github.com:httpie/cli.git

In [ ]:
# First, we need to read all python files in the project.
import os

path = "./cli/httpie/**/*.py"

from glob import glob

for filepath in glob(path, recursive=True):
    print(filepath)

In [ ]:
pyfiles = list(glob(path, recursive=True))
len(pyfiles)

## Get AST one-by-one

In [ ]:
import ast

def getAST(path: str):
    with open(path, "r") as f:
        source = f.read()
        
    return ast.parse(source)

In [ ]:
myast = getAST(pyfiles[0])
myast

## Detect an inheritance syntax.

In [ ]:
from ast import NodeVisitor

class SuperClassNameVisitor(NodeVisitor):
    def __init__(self):
        super().__init__()
        self.names = []
    
    def visit_Name(self, node):
        self.names.append(node.id)
        return super().generic_visit(node)

class ClassVisitor(NodeVisitor):
    def __init__(self):
        super().__init__()
        self.gen_dict = {}
        
    def visit_ClassDef(self, node):
        namevisitor = SuperClassNameVisitor()
        for base in node.bases:
            namevisitor.visit(base)
        self.gen_dict[node.name] = { "parents": namevisitor.names }
#         print(node.name, list([base.id for base in node.bases]))
        return super().generic_visit(node)
        
myvisitor = ClassVisitor()
myvisitor.visit(myast)

## Store generalization relationships in a data structure

In [ ]:
myvisitor = ClassVisitor()

for pyfile in pyfiles:
    myast = getAST(pyfile)
    myvisitor.visit(myast)
    
myvisitor.gen_dict

# Association

## Composition

## Aggregation